In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import glob

In [6]:
#majors2020 = pd.read_table("data/uc.berkeley.student.majors.2020.txt")
df = pd.DataFrame()
files = glob.glob("data/*.txt")
course_data = glob.glob("data/uc.berkeley.student.course.data.*.txt")
majors = glob.glob("data/uc.berkeley.student.majors.*.txt")
pathways = glob.glob("data/uc.berkeley.stem.*.txt")


In [28]:
stem = pd.read_table(files[24]).groupby(['year','ethnic.l1']).count()
#stem = pd.DataFrame(stem)
stem

2786    2786   

                                ethnic.l2  ethnic.l3  probation.year.1  \
year ethnic.l1                                                           
2007 Asian/Pacific Islander          3006       3006              3006   
     International                    292        292               292   
     Underrepresented Minority       1193       1193              1193   
     White/Other                     2947       2947              2947   
2008 Asian/Pacific Islander          2940       2940              2940   
     International                    527        527               527   
     Underrepresented Minority       1160       1160              1160   
     White/Other                     2888       2888              2888   
2009 Asian/Pacific Islander          2887       2887              2887   
     International                    605        605               605   
     Underrepresented Minority       1235       1235              1235   
     White/Other                     2984       2984              2984   
2010 Asian/Pacific Islander          2830       2830              2830   
     International                    778        778               778   
     Underrepresented Minority       1185       1185              1185   
     White/Other                     2831       2831              2831   
2011 Asian/Pacific Islander          2922       2922              2922   
     International                   1074       1074              1074   
     Underrepresented Minority       1386       1386              1386   
     White/Other                     2622       2622              2622   
2012 Asian/Pacific Islander          2883       2883              2883   
     International                   1023       1023              1023   
     Underrepresented Minority       1350       1350              1350   
     White/Other                     2374       2374              2374   
2013 Asian/Pacific Islander          2954       2954              2954   
     International                   1413       1413              1413   
     Underrepresented Minority       1353       1353              1353   
     White/Other                     2448       2448              2448   
2014 Asian/Pacific Islander          3341       3341              3341   
     International                   1219       1219              1219   
     Underrepresented Minority       1707       1707              1707   
     White/Other                     2682       2682              2682   
2015 Asian/Pacific Islander          3072       3072              3072   
     International                   1135       1135              1135   
     Underrepresented Minority       1510       1510              1510   
     White/Other                     2527       2527              2527   
2016 Asian/Pacific Islander          3392       3392              3392   
     International                   1297       1297              1297   
     Underrepresented Minority       1750       1750              1750   
     White/Other                     2786       2786              2786   

                                GradYr  ...  degree.department.2  \
year ethnic.l1                          ...                        
2007 Asian/Pacific Islander       2871  ...                  520   
     International                 267  ...                   39   
     Underrepresented Minority    1055  ...                  145   
     White/Other                  2737  ...                  347   
2008 Asian/Pacific Islander       2769  ...                  408   
     International                 490  ...                   98   
     Underrepresented Minority    1002  ...                  136   
     White/Other                  2679  ...                  339   
2009 Asian/Pacific Islander       2724  ...                  428   
     International                 556  ...                  101   
     Underrepresented Minority    1090  ...               

In [20]:
stem

count
ethnic.l1                 year       
Asian/Pacific Islander    2007   3006
                          2008   2940
                          2009   2887
                          2010   2830
                          2011   2922
                          2012   2883
                          2013   2954
                          2014   3341
                          2015   3072
                          2016   3392
International             2007    292
                          2008    527
                          2009    605
                          2010    778
                          2011   1074
                          2012   1023
                          2013   1413
                          2014   1219
                          2015   1135
                          2016   1297
Underrepresented Minority 2007   1193
                          2008   1160
                          2009   1235
                          2010   1185
                          2011   1386
                          2012   1350
                          2013   1353
                          2014   1707
                          2015   1510
                          2016   1750
White/Other               2007   2947
                          2008   2888
                          2009   2984
                          2010   2831
                          2011   2622
                          2012   2374
                          2013   2448
                          2014   2682
                          2015   2527
                          2016   2786

In [48]:
stem.columns

Index(['ppsk', 'count', 'term', 'entry', 'residency', 'gender', 'ethnic.l2',
       'ethnic.l3', 'probation.year.1', 'GradYr',
       ...
       'degree.department.2', 'degree.major.2', 'degree.college.3',
       'degree.division.3', 'degree.department.3', 'degree.major.3',
       'degree.college.4', 'degree.division.4', 'degree.department.4',
       'degree.major.4'],
      dtype='object', length=106)

In [14]:
plt.plot('year', 'count', data = stem)

ValueError: x and y must have same first dimension, but have shapes (1,) and (40,)